In [78]:
%pip install langdetect

Note: you may need to restart the kernel to use updated packages.


# Import các thư viện cần thiết

In [79]:
import os
from tqdm import tqdm
from typing import List
import pandas as pd

from langdetect import detect
from langdetect import DetectorFactory

# Display all columns
pd.set_option('display.max_columns', None)

# 1. Tiền xử lý dữ liệu transcription từ video

Liệt kê đường dẫn đến các file .csv cần thiết

In [80]:
def list_file_types(directory: str, file_extension: str) -> List[str]:
    """ List all files with a specific extension in a directory.

    Args:
        directory (str): Directory path.
        file_extension (str): File extension.

    Returns:
        List[str]: List of file paths.
    """

    file_list: List[str] = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(file_extension):
                file_list.append(os.path.join(root, file))
    return file_list


transcription_files = list_file_types("../../data/VideoTranscription", ".csv")
transcription_files[:5]

['../../data/VideoTranscription\\Kaggle\\0000000000000000_audio_transcription_12800_13000.csv',
 '../../data/VideoTranscription\\Kaggle\\0000000000000000_audio_transcription_13000_13200.csv',
 '../../data/VideoTranscription\\Kaggle\\0000000000000000_audio_transcription_13200_13400.csv',
 '../../data/VideoTranscription\\Kaggle\\0000000000000000_audio_transcription_13400_13600.csv',
 '../../data/VideoTranscription\\Kaggle\\0000000000000000_audio_transcription_13600_13800.csv']

Đọc lần lượt các file .csv thành các DataFrame và lưu vào một list. Đồng thời tạo thêm một cột `url` chứa đường dẫn đến TikTok video tương ứng với mỗi transcription.


In [81]:
# Create a function to format TikTok URLs
def format_tiktok_url(author_id: str, video_id: str) -> str:
    """ Format TikTok URL.

    Args:
        author_id (str): ID of the author.
        video_id (str): ID of the video.

    Returns:
        str: TikTok URL.
    """

    return f"https://www.tiktok.com/@{author_id}/video/{video_id}"

In [82]:
df_list: List[pd.DataFrame] = []

for transcription_file in tqdm(transcription_files):
    # Load the CSV file
    df = pd.read_csv(transcription_file)

    # Apply the function to create or update the URL column
    df['url'] = df.apply(lambda row: format_tiktok_url(
        row['author_id'], row['video_id']), axis=1)

    # Select columns order
    df[['author_id', 'video_id', 'video_transcription', 'url']]

    # Append the DataFrame to the list
    df_list.append(df)

100%|██████████| 75/75 [00:01<00:00, 62.41it/s] 


Merge các DataFrame trong list thành một DataFrame duy nhất.

In [83]:
# Concatenate all DataFrames in the list
transcription_df = pd.concat(df_list, ignore_index=True)
transcription_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33501 entries, 0 to 33500
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   author_id            33501 non-null  int64 
 1   video_id             33501 non-null  int64 
 2   video_transcription  28929 non-null  object
 3   url                  33501 non-null  object
dtypes: int64(2), object(2)
memory usage: 1.0+ MB


**Nhận xét:**

- Ta thấy có nhiều video không có transcription, ta sẽ loại bỏ các dòng này.

Loại bỏ các dòng bị thiếu dữ liệu.

In [84]:
# Remove rows with missing values
transcription_df = transcription_df.dropna(
    axis='index', how='any'
)

# Reset index after removing rows
transcription_df = transcription_df.reset_index(drop=True)

transcription_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28929 entries, 0 to 28928
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   author_id            28929 non-null  int64 
 1   video_id             28929 non-null  int64 
 2   video_transcription  28929 non-null  object
 3   url                  28929 non-null  object
dtypes: int64(2), object(2)
memory usage: 904.2+ KB


Loại bỏ các dòng có transcription ít hơn 50 từ (word).

In [85]:
# Remove rows with less than 50 words in the transcription
transcription_df = transcription_df[
    transcription_df['video_transcription'].str.split().apply(len) >= 50
]

# Reset index after removing rows
transcription_df = transcription_df.reset_index(drop=True)

transcription_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23825 entries, 0 to 23824
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   author_id            23825 non-null  int64 
 1   video_id             23825 non-null  int64 
 2   video_transcription  23825 non-null  object
 3   url                  23825 non-null  object
dtypes: int64(2), object(2)
memory usage: 744.7+ KB


Loại bỏ các dòng có transcription không phải tiếng Việt.

In [86]:
# Create a new column for the detected language
transcription_df["language"] = ["error"] * len(transcription_df)

# Detect the language of the transcription
for index in tqdm(range(len(transcription_df))):
    try:
        DetectorFactory.seed = 0
        transcription_df.loc[index, "language"] = detect(
            transcription_df.loc[index, "video_transcription"])
    except:
        transcription_df.loc[index, "language"] = "error"

100%|██████████| 23825/23825 [01:40<00:00, 237.07it/s]


Số lượng transcription bằng tiếng Việt còn lại sau khi loại bỏ các dòng không hợp lệ.

In [87]:
sum(transcription_df['language'] == 'vi')

23765

In [88]:
# Remove rows with non-Vietnamese transcriptions
transcription_df = transcription_df[
    transcription_df['language'] == 'vi'
]

# Reset index after removing rows
transcription_df = transcription_df.reset_index(drop=True)

transcription_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23765 entries, 0 to 23764
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   author_id            23765 non-null  int64 
 1   video_id             23765 non-null  int64 
 2   video_transcription  23765 non-null  object
 3   url                  23765 non-null  object
 4   language             23765 non-null  object
dtypes: int64(2), object(3)
memory usage: 928.4+ KB


In [89]:
transcription_df.sample(n=5)

,author_id,video_id,video_transcription,url,language
20527,7080408427277190145,7464518477731204360,Bây giờ là 2 giờ chiều nhưng mà tiến hành vào ...,https://www.tiktok.com/@7080408427277190145/vi...,vi
6049,6908933207383639041,7420682548054936839,đi từ 4 giờ đúng không có nhà ai như nhà mình ...,https://www.tiktok.com/@6908933207383639041/vi...,vi
3242,6844492227462267905,7368040029412674832,136 món hải sản như thế này chưa tới một củ mọ...,https://www.tiktok.com/@6844492227462267905/vi...,vi
18096,7255941113726321666,7415952588656413960,Xin chào mọi người Hôm nay ăn một mẹ đồ ăn tha...,https://www.tiktok.com/@7255941113726321666/vi...,vi
19200,6573106305933475841,7470143947986308360,Hồi chiều tới giờ mà còn đọc được cái vụ này c...,https://www.tiktok.com/@6573106305933475841/vi...,vi


Chuyển kiểu dữ liệu của tất cả các cột thành kiểu `str`

In [90]:
# Convert data type of all columns to string
transcription_df = transcription_df.astype(str)
transcription_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23765 entries, 0 to 23764
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   author_id            23765 non-null  object
 1   video_id             23765 non-null  object
 2   video_transcription  23765 non-null  object
 3   url                  23765 non-null  object
 4   language             23765 non-null  object
dtypes: object(5)
memory usage: 928.4+ KB


Loại bỏ các dòng bị trùng lặp. Dựa trên cột `video_id` để xác định dòng nào là trùng lặp. Với các dòng trùng lặp, ta chỉ giữ lại dòng đầu tiên.

In [91]:
# Display number of rows before deduplication
print(f"Before: {len(transcription_df)}")

# Remove duplicates based on video_id column, keeping the first occurrence
transcription_df = transcription_df.drop_duplicates(
    subset=["video_id"], keep='first'
)

# Reset index after deduplication
transcription_df = transcription_df.reset_index(drop=True)

# Display number of rows after deduplication
print(f"After: {len(transcription_df)}")

Before: 23765
After: 23099


# 2. Tiền xử lý dữ liệu thống kê từ video

Đọc dữ liệu thống kê video thành DataFrame.

In [92]:
video_info_df = pd.read_csv("../../data/interim/video_info.csv", low_memory=False)
video_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32603 entries, 0 to 32602
Columns: 136 entries, AIGCDescription to audio_to_text
dtypes: bool(20), float64(37), int64(36), object(43)
memory usage: 29.5+ MB


Loại bỏ các cột có tỷ lệ thiếu dữ liệu lớn hơn 50%

In [93]:
# Calculate the missing ratio
missing_ratio = video_info_df.isna().sum() / len(video_info_df)

# Sort the missing ratio in descending order
missing_ratio = missing_ratio.sort_values(ascending=False)

# Display first 5 rows
missing_ratio.head()

AIGCDescription               1.0
backendSourceEventTracking    1.0
music.coverThumb              1.0
music.coverMedium             1.0
music.coverLarge              1.0
dtype: float64

In [94]:
# Remove columns with missing ratios greater than 0.50
video_info_df = video_info_df.dropna(
    axis='columns', thresh=0.50 * len(video_info_df)
)

In [95]:
# Calculate the missing ratio
missing_ratio = video_info_df.isna().sum() / len(video_info_df)

# Sort the missing ratio in descending order
missing_ratio = missing_ratio.sort_values(ascending=False)

# Display the missing ratio
for column, ratio in missing_ratio[:5].items():
    print(f"{column:50}:{ratio:8.2%}")

duetEnabled                                       :  20.29%
stitchEnabled                                     :  19.87%
video.claInfo.originalLanguageInfo.languageID     :  18.91%
video.claInfo.originalLanguageInfo.languageCode   :  18.91%
video.claInfo.originalLanguageInfo.language       :  18.91%


In [96]:
# video_info_df.info()

Vì các cột có tên bắt đầu với `stats.*` chứa cùng thông tin với các cột có tên bắt đầu với `statsV2.*`, nhưng không có thông tin về `repostCount` như `statsV2.*`. Nên ta sẽ loại bỏ các cột có tên bắt đầu với `stats.*`.


In [97]:
# Remove columns starting with "stats."
video_info_df = video_info_df[
    [column for column in video_info_df.columns
            if not column.startswith("stats.")]
]

# video_info_df.info()

Xóa các cột bắt đầu với `video.claInfo.originalLanguageInfo.*` vì chúng chứa thông tin không cần thiết.


In [98]:
# Remove columns starting with "video.claInfo.originalLanguageInfo."
video_info_df = video_info_df[
    [column for column in video_info_df.columns
            if not column.startswith("video.claInfo.originalLanguageInfo.")]
]

# video_info_df.info()

Tạo một cột chứa danh sách các `hashtag` được trích xuất từ mô tả video. Và tính số lượng hashtag trong mỗi video.

In [99]:
# Replace missing values in "desc" column with an empty string
video_info_df["desc"] = video_info_df["desc"].fillna("")
video_info_df["desc"] = video_info_df["desc"].astype(str)
video_info_df["desc"] = video_info_df["desc"].str.strip()

# Create a new column for the hashtags
# and the number of hashtags in each video
video_info_df["hashtags"] = [""] * len(video_info_df)
video_info_df["num_hashtags"] = [0] * len(video_info_df)

# Extract hashtags from the "desc" column
# and Get the number of hashtags in each video
for index in tqdm(range(len(video_info_df))):
    # Get the description of the video
    description = video_info_df.loc[index, "desc"].strip().lower()

    if description:
        # Remove emojis
        description = description.encode('ascii', 'ignore').decode('ascii')

        # Add a space before all "#" characters
        description = description.replace("#", " #")

        # Find all strings starting with "#" and followed by a word
        hashtags = [word[1:] for word in description.split()
                    if word.startswith("#")]

        # Extract hashtags from the description
        video_info_df.loc[index, "hashtags"] = ",".join(hashtags).strip()

        # Get the number of hashtags
        video_info_df.loc[index, "num_hashtags"] = len(hashtags)
    else:
        video_info_df.loc[index, "hashtags"] = ""
        video_info_df.loc[index, "num_hashtags"] = 0

100%|██████████| 32603/32603 [00:06<00:00, 4718.58it/s]


In [100]:
video_info_df[["hashtags", "num_hashtags"]].sample(n=5)

,hashtags,num_hashtags
10953,"saigon,didausaigon,binhthanh,ancungtiktok,lear...",5
843,"anchoivungtau,anchoivungtau72,xuhuong,fyp,duli...",10
16288,"learnontiktok,ancungtiktok,xuhuong,tiktokfood,...",6
27993,"tieumanthau,learnontiktok,vtmgr,ancungtiktok,c...",13
13197,"onganhthichnauan,fyp,cooking,vanmonngonbungvi,...",5


In [101]:
video_info_df['hashtags'] = video_info_df['hashtags'].apply(lambda x: x.split(',') if isinstance(x, str) and x.strip() else [])
video_info_df["hashtags"] = video_info_df["hashtags"].apply(lambda x: x if isinstance(x, list) else [])


In [102]:
video_info_df['createTime'] = pd.to_datetime(video_info_df['createTime'], unit='s')


# 3. Nối dữ liệu transcription và dữ liệu thống kê

Merge hai DataFrame transcription và DataFrame thống kê thành một DataFrame duy nhất. Sử dụng videoId làm khóa ngoại.

In [103]:
# Convert data type of video.id to string
video_info_df["video.id"] = video_info_df["video.id"].astype(str)
transcription_df["video_id"] = transcription_df["video_id"].astype(str)

# Merge the DataFrames
merged_df = pd.merge(
    left=video_info_df,
    right=transcription_df,
    how="inner",
    left_on="video.id",
    right_on="video_id"
)

# Reset index after merging
merged_df = merged_df.reset_index(drop=True)

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23099 entries, 0 to 23098
Data columns (total 84 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   CategoryType                     23099 non-null  int64         
 1   author.commentSetting            23099 non-null  int64         
 2   author.downloadSetting           23099 non-null  int64         
 3   author.duetSetting               23099 non-null  int64         
 4   author.ftc                       23099 non-null  bool          
 5   author.id                        23099 non-null  int64         
 6   author.isADVirtual               23099 non-null  bool          
 7   author.isEmbedBanned             23099 non-null  bool          
 8   author.nickname                  23099 non-null  object        
 9   author.openFavorite              23099 non-null  bool          
 10  author.privateAccount            23099 non-null  bool     

Kiểm tra các hàng bị trùng lặp và loại bỏ chúng.

In [104]:
num_duplicates = merged_df.duplicated(subset=["video_id"]).sum()
num_duplicates

0

Loại bỏ các cột có cùng thông tin với các cột khác: `author_id`, `video_id`, v.v.

In [105]:
# Remove redundant columns
redundant_columns = [
    "video_id", "author_id", "language"
]
merged_df = merged_df.drop(columns=redundant_columns, axis=1,
                           errors='ignore')

In [106]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23099 entries, 0 to 23098
Data columns (total 81 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   CategoryType                     23099 non-null  int64         
 1   author.commentSetting            23099 non-null  int64         
 2   author.downloadSetting           23099 non-null  int64         
 3   author.duetSetting               23099 non-null  int64         
 4   author.ftc                       23099 non-null  bool          
 5   author.id                        23099 non-null  int64         
 6   author.isADVirtual               23099 non-null  bool          
 7   author.isEmbedBanned             23099 non-null  bool          
 8   author.nickname                  23099 non-null  object        
 9   author.openFavorite              23099 non-null  bool          
 10  author.privateAccount            23099 non-null  bool     

In [107]:
merged_df.sample(n=2)

,CategoryType,author.commentSetting,author.downloadSetting,author.duetSetting,author.ftc,author.id,author.isADVirtual,author.isEmbedBanned,author.nickname,author.openFavorite,author.privateAccount,author.relation,author.secUid,author.secret,author.signature,author.stitchSetting,author.uniqueId,author.verified,authorStats.diggCount,authorStats.followerCount,authorStats.followingCount,authorStats.friendCount,authorStats.heart,authorStats.heartCount,authorStats.videoCount,collected,createTime,desc,digged,diversificationId,duetDisplay,duetEnabled,forFriend,id,isAd,itemCommentStatus,item_control.can_repost,music.authorName,music.duration,music.id,music.isCopyrighted,music.original,music.title,officalItem,originalItem,privateItem,secret,shareEnabled,statsV2.collectCount,statsV2.commentCount,statsV2.diggCount,statsV2.playCount,statsV2.repostCount,statsV2.shareCount,stitchDisplay,stitchEnabled,textLanguage,textTranslatable,video.VQScore,video.bitrate,video.claInfo.enableAutoCaption,video.claInfo.hasOriginalAudio,video.codecType,video.definition,video.duration,video.encodedType,video.format,video.height,video.id,video.ratio,video.videoID,video.videoQuality,video.volumeInfo.Loudness,video.volumeInfo.Peak,video.width,collectTime,video.claInfo.captionsType,hashtags,num_hashtags,video_transcription,url
19937,111,0,3,0,False,6908933207383639041,False,False,Tiểu Màn Thầu,False,False,0,MS4wLjABAAAA3wojSkfJjQnNK5vyHC5eLhP8A-GpvIp5VU...,False,Contact for work: 093 4307361 (Lan Anh)\r\n--\...,0,tieumanthaune,True,25100,3300000,168,0,111500000,111500000,1104,False,2024-02-21 15:06:15,💯 Tự nhiên cái trời nóng ghê! #tieumanthau #Le...,False,10085.0,0,True,False,7338071754339994887,False,0,True,dama,486.0,6925885701440178178,False,False,Cute background music for kids(951274),False,False,False,False,True,430,57,18200,345600,0,71,0,True,vi,True,69.27,1502891.0,True,True,h264,540p,59,normal,mp4,1024,7338071754339994887,540p,v10044g50000cnb123vog65q0fhqtvh0,normal,-15.0,0.60256,576,1741176068,1.0,"[tieumanthau, learnontiktok, vtmgr, ancungtikt...",10,cách làm mùa hè tới rồi hay sao nóng quá nha m...,https://www.tiktok.com/@6908933207383639041/vi...
16905,111,0,0,0,False,6859343544352687105,False,False,Đỗ Phượng Vỹ,False,False,0,MS4wLjABAAAAVSMqrednlizzyF26dxOdZvQSoVWurvluz-...,False,Đặt sữa hạt ở phở: Đỗ Phượng Vỹ\r\nEmail: Doph...,0,phuongvy_1105,False,1643,2100000,113,0,57900000,57900000,1006,False,2024-05-06 07:18:14,Mùa sầu riêng tới rồi mn ơi 🤭 #phuongvy_1105 #...,False,10039.0,0,True,False,7365782525785951495,False,0,True,Đỗ Phượng Vỹ,145.0,7365782682250644225,False,False,nhạc nền - Phượng Vỹ,False,False,False,False,True,1237,749,60700,1500000,0,2186,0,True,vi,True,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,7365782525785951495,NaN,NaN,NaN,NaN,NaN,0,1741177527,NaN,"[phuongvy_1105, ancungtiktok, anngonnaugon, tr...",4,Tổng cộng là có bốn cái trong đó thì có 3 trái...,https://www.tiktok.com/@6859343544352687105/vi...


In [108]:
merged_df

,CategoryType,author.commentSetting,author.downloadSetting,author.duetSetting,author.ftc,author.id,author.isADVirtual,author.isEmbedBanned,author.nickname,author.openFavorite,author.privateAccount,author.relation,author.secUid,author.secret,author.signature,author.stitchSetting,author.uniqueId,author.verified,authorStats.diggCount,authorStats.followerCount,authorStats.followingCount,authorStats.friendCount,authorStats.heart,authorStats.heartCount,authorStats.videoCount,collected,createTime,desc,digged,diversificationId,duetDisplay,duetEnabled,forFriend,id,isAd,itemCommentStatus,item_control.can_repost,music.authorName,music.duration,music.id,music.isCopyrighted,music.original,music.title,officalItem,originalItem,privateItem,secret,shareEnabled,statsV2.collectCount,statsV2.commentCount,statsV2.diggCount,statsV2.playCount,statsV2.repostCount,statsV2.shareCount,stitchDisplay,stitchEnabled,textLanguage,textTranslatable,video.VQScore,video.bitrate,video.claInfo.enableAutoCaption,video.claInfo.hasOriginalAudio,video.codecType,video.definition,video.duration,video.encodedType,video.format,video.height,video.id,video.ratio,video.videoID,video.videoQuality,video.volumeInfo.Loudness,video.volumeInfo.Peak,video.width,collectTime,video.claInfo.captionsType,hashtags,num_hashtags,video_transcription,url
0,111,0,0,0,False,7128234498731803674,False,False,1 phút Sài Gòn,False,False,0,MS4wLjABAAAA3QoEqn5xhIihiigzt7wJcPLXYYrGZb60yQ...,False,Lụm lặt những điều nhỏ nhỏ dễ thương ở Sài Gòn.,0,1phutsaigon,False,2128,198700,47,0,4100000,4100000,847,False,2025-02-13 14:08:47,"Phát hiện chiếc quán có nóc nhà cực chill, nơi...",False,10085.0,0,True,False,7470905424930262289,False,0,True,1 phút Sài Gòn,28.0,7470905515782966033,True,False,nhạc nền - 1 phút Sài Gòn,False,False,False,False,True,12283,414,32100,735300,0,10200,0,True,vi,True,72.87,2660641.0,True,True,h264,540p,28,normal,mp4,1024,7470905424930262289,540p,v1c044g50000cumvomnog65hhpslq940,normal,-7.2,1.00000,576,1741176061,NaN,"[1phutsaigon, saigon, saigondidau, cafesaigon,...",7,cuối cùng mà mình thấy nhau và rồi tháng mấy đ...,https://www.tiktok.com/@7128234498731803674/vi...
1,105,0,0,0,False,7128234498731803674,False,False,1 phút Sài Gòn,False,False,0,MS4wLjABAAAA3QoEqn5xhIihiigzt7wJcPLXYYrGZb60yQ...,False,Lụm lặt những điều nhỏ nhỏ dễ thương ở Sài Gòn.,0,1phutsaigon,False,2128,198700,47,0,4100000,4100000,847,False,2025-01-28 12:08:58,Tổng hợp các địa điểm du Xuân chụp hình Tết ở ...,False,10014.0,0,True,False,7464937169858170129,False,0,True,1 phút Sài Gòn,25.0,7464937228537170689,True,False,nhạc nền - 1 phút Sài Gòn,False,False,False,False,True,82,354,694,26100,0,68,0,True,vi,True,73.22,3619524.0,True,True,h264,540p,25,normal,mp4,1024,7464937169858170129,540p,v1c044g50000cucch7nog65hlk5usma0,normal,-8.3,1.00000,576,1741176061,NaN,"[1phutsaigon, saigon, saigondidau, duxuan, tet...",5,Nghe xuân sang thấy trong lòng mình chứa chan ...,https://www.tiktok.com/@7128234498731803674/vi...
2,111,0,0,0,False,7128234498731803674,False,False,1 phút Sài Gòn,False,False,0,MS4wLjABAAAA3QoEqn5xhIihiigzt7wJcPLXYYrGZb60yQ...,False,Lụm lặt những điều nhỏ nhỏ dễ thương ở Sài Gòn.,0,1phutsaigon,False,2128,198700,47,0,4100000,4100000,847,False,2025-01-19 14:15:33,Phát hiện khu vườn ngập tràn sắc xuân xinh xắn...,False,10085.0,0,True,False,7461630006834367745,False,0,True,1 phút Sài Gòn,28.0,7461630131497683729,False,False,nhạc nền - 1 phút Sài Gòn,False,False,False,False,True,251,25,4477,41300,0,157,0,True,vi,True,72.58,3178076.0,True,True,h264,540p,28,normal,mp4,1024,7461630006834367745,540p,v14044g50000cu6ghuvog65s16m66m0g,normal,-7.0,1.00000,576,1741176062,NaN,"[1phutsaigon, saigon, saigondidau, halacoffee,...",6,bồi hồi liên kết sẽ quay về thăm quê em xuân đ...,https://www.tiktok.com/@7128234498731803674/vi...
3,111,0,0,0,False,7128234498731803674,False,False,1 phút Sài Gòn,False,False,0,MS4wLjABAAAA3QoEqn5xhIihiigzt7wJcPLXYYrGZb60yQ...,False,Lụm lặt những điều nhỏ nhỏ dễ thương ở Sài Gòn.,0,1phutsaigon,Fal

# Export to Merge

Lưu DataFrame đã xử lý vào file `merged_data.csv`.

In [109]:
# merged_df.to_csv("data/processed/merged_data.csv", index=False)

In [110]:
# merged_df.to_parquet("data/processed/merged_data.parquet", index=False)

# ========================================

In [111]:
merged_df.columns

Index(['CategoryType', 'author.commentSetting', 'author.downloadSetting',
       'author.duetSetting', 'author.ftc', 'author.id', 'author.isADVirtual',
       'author.isEmbedBanned', 'author.nickname', 'author.openFavorite',
       'author.privateAccount', 'author.relation', 'author.secUid',
       'author.secret', 'author.signature', 'author.stitchSetting',
       'author.uniqueId', 'author.verified', 'authorStats.diggCount',
       'authorStats.followerCount', 'authorStats.followingCount',
       'authorStats.friendCount', 'authorStats.heart',
       'authorStats.heartCount', 'authorStats.videoCount', 'collected',
       'createTime', 'desc', 'digged', 'diversificationId', 'duetDisplay',
       'duetEnabled', 'forFriend', 'id', 'isAd', 'itemCommentStatus',
       'item_control.can_repost', 'music.authorName', 'music.duration',
       'music.id', 'music.isCopyrighted', 'music.original', 'music.title',
       'officalItem', 'originalItem', 'privateItem', 'secret', 'shareEnabled',
     

In [122]:
import plotly.express as px
import plotly.graph_objects as go


# Visual for 

In [141]:
# Define video duration categories
bins = [0, 10, 30, 60, 90, 120, 180, 300, 600, float("inf")]
labels = ["<10s", "10-30s", "30-60s", "60-90s", "90-120s", "2 mins", "3-5 mins", "5-10 mins", ">10 mins"]

# Categorize video durations
video_info_df["video_duration_category"] = pd.cut(video_info_df["video.duration"], bins=bins, labels=labels, right=False)

# Count number of videos in each category
duration_counts = video_info_df["video_duration_category"].value_counts().reindex(labels)

# Calculate total views per video duration category
views_per_category = video_info_df.groupby("video_duration_category")["statsV2.playCount"].sum().reindex(labels)

# Create the figure
fig = go.Figure()

# Add bar chart for number of videos
fig.add_trace(go.Bar(
    x=duration_counts.index,
    y=duration_counts.values,
    name="Number of Videos",
    marker_color="blue",
    yaxis="y1"
))

# Add line chart for total views
fig.add_trace(go.Scatter(
    x=views_per_category.index,
    y=views_per_category,
    name="Total Views",
    mode="lines+markers",
    marker=dict(color="red", size=8),
    line=dict(width=2),
    yaxis="y2"
))

# Update layout
fig.update_layout(
    title="Video Duration vs. Total Views",
    xaxis_title="Video Duration Category",
    
    # Left Y-Axis (Number of Videos)
    yaxis=dict(
        title="Number of Videos",
        side="left",
        showgrid=False
    ),
    
    # Right Y-Axis (Total Views)
    yaxis2=dict(
        title="Total Views",
        side="right",
        overlaying="y",
        showgrid=False
    ),
    
    margin=dict(l=60, r=60, t=50, b=50),
    legend=dict(x=0.1, y=1.1),
)

# Show the figure
fig.show()


C:\Users\029at\AppData\Local\Temp\ipykernel_17884\1644838173.py:12: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [144]:
import pandas as pd
import plotly.express as px

# Define video duration categories
bins = [0, 10, 30, 60, 90, 120, 180, 300, 600, float("inf")]
labels = ["<10s", "10-30s", "30-60s", "60-90s", "90-120s", "2 mins", "3-5 mins", "5-10 mins", ">10 mins"]

# Categorize video durations
video_info_df["video_duration_category"] = pd.cut(video_info_df["video.duration"], bins=bins, labels=labels, right=False)

# Melt dataframe to include views, likes, comments, and shares
engagement_df = video_info_df.melt(
    id_vars=["video_duration_category"],
    value_vars=["statsV2.playCount", "statsV2.diggCount", "statsV2.commentCount", "statsV2.shareCount"],
    var_name="Engagement Type",
    value_name="Count"
)

# Rename engagement types for better readability
engagement_df["Engagement Type"] = engagement_df["Engagement Type"].replace({
    "statsV2.playCount": "Views",
    "statsV2.diggCount": "Likes",
    "statsV2.commentCount": "Comments",
    "statsV2.shareCount": "Shares"
})

# Create scatter plot
fig_scatter = px.scatter(
    engagement_df,
    x="video_duration_category",
    y="Count",
    color="Engagement Type",
    title="Video Duration vs. Engagement (Views, Likes, Comments, Shares)",
    labels={"video_duration_category": "Video Duration Category", "Count": "Engagement Count"},
    category_orders={"video_duration_category": labels}
)

# Show figure
fig_scatter.show()
